In [ ]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sales_train=pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
test=pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
sample_submission=pd.read_csv('../input/competitive-data-science-predict-future-sales/sample_submission.csv')
items=pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
items_category=pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
shops=pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')

In [ ]:
print("shape info of sales_train: {}".format(sales_train.shape))
print("shape info of test: {}".format(test.shape))
print("shape info of sample submission {}".format(sample_submission.shape))
print("shape info of items {}".format(items.shape))
print("shape info of items category {}".format(items_category.shape))
print("shape info of shops {}".format(shops.shape))

In [ ]:
sales_train.info()

In [ ]:
sales_train.head()

Date block simply represents the month  -1

In [ ]:
print(sales_train.head())
print("===============================================================")
print(test.head())
print("===============================================================")
print(items.head())
print("===============================================================")
print(items_category.head())
print("===============================================================")
print(shops.head())
print("===============================================================")

In [ ]:
# from statsmodels.tsa.seasonal import seasonal_decompose
# from dateutil.parser import parse


# # Multiplicative Decomposition 
# multiplicative_decomposition = seasonal_decompose(sales_train['item_cnt_day'], model='multiplicative', period=30)

# # Additive Decomposition
# additive_decomposition = seasonal_decompose(sales_train['item_cnt_day'], model='additive', period=30)

# # Plot
# plt.rcParams.update({'figure.figsize': (16,12)})
# multiplicative_decomposition.plot().suptitle('Multiplicative Decomposition', fontsize=16)
# plt.tight_layout(rect=[0, 0.03, 1, 0.95])

# additive_decomposition.plot().suptitle('Additive Decomposition', fontsize=16)
# plt.tight_layout(rect=[0, 0.03, 1, 0.95])

# plt.show()

In [ ]:
sales_train = sales_train.join(items, on = 'item_id', lsuffix = '_caller',rsuffix = '_other')

In [ ]:
sales_train.drop(["item_id_other"],axis = 1, inplace = True)

In [ ]:
sales_train.rename(columns={'item_id_caller': 'item_id'},
          inplace=True, errors='raise')

In [ ]:
sales_train['tot_item'] = sales_train['item_price']*sales_train['item_cnt_day']

In [ ]:
# import re
# sales_train['day']=[ re.split('\.',sales_train['date'][i])[0] for i in range(len(sales_train['date']))]
# sales_train['month']=[ re.split('\.',sales_train['date'][i])[1] for i in range(len(sales_train['date']))]
# sales_train['year']=[ re.split('\.',sales_train['date'][i])[2] for i in range(len(sales_train['date']))]

**Let's add a column for the region name**

In [ ]:
shops[['region','name']] = shops['shop_name'].str.split(" ",1,expand = True)

# 🔎EDA

In [ ]:
shops.groupby('region',as_index=False)['region'].size().plot(x = 'region',kind='bar');
plt.title('Number of shops in each region')


**We can see that "Mockba" has the most stores ... maybe that means that it has the most sales ??**

In [ ]:
sales_train.head()

In [ ]:
sales_train = sales_train.merge(shops,how='left',on='shop_id')

In [ ]:
sales_train.head()


In [ ]:
sales_train.columns

In [ ]:
sales_train=sales_train[['date','date_block_num','shop_id','item_id','item_price','item_cnt_day','item_category_id','tot_item','region']]

In [ ]:
sales_train['date'] = pd.to_datetime(sales_train['date'])

In [ ]:
sales_train.set_index('date', inplace=True)


In [ ]:
sales_train.head()

In [ ]:
plt.figure(figsize=(50,30))

sales_train['item_cnt_day'].resample('M').agg(['sum']).plot(color='red')
plt.title('Total Items sold each month')
plt.xlabel('month')
# plt.ylabel('item count')
plt.show()

In [ ]:
plt.figure(figsize=(50,30))

sales_train['item_cnt_day'].resample('Q').agg(['sum']).plot(color='red')
plt.title('Total Items sold each quarter')
plt.xlabel('month')
# plt.ylabel('item  count')
plt.show()

We can clearly see that the sales are peaking at the 

**Is there any missing values?**

In [ ]:
print(sales_train.isnull().sum())
print()
print(test.isnull().sum())
print()
print(items.isnull().sum())
print()
print(items_category.isnull().sum())

In [ ]:
sales_train.columns

In [ ]:
plt.figure(figsize=(25,10))
plt.bar(sales_train.groupby('shop_id', as_index=False).agg({"item_cnt_day": "sum"}).index,sales_train.groupby('shop_id', as_index=False).agg({"item_cnt_day": "sum"})['item_cnt_day'])
plt.xlabel("Store", size = 15) 
# plt.ylabel("Sum of sales", size = 15) 
plt.title("Sum of sales per store" , size = 20)

**The sales distribution is generally healthy, we can see that store 31 and store 25 have the sales**

In [ ]:
plt.figure(figsize=(25,10))
plt.bar(sales_train.groupby('item_category_id', as_index=False).agg({"item_cnt_day": "sum"}).index,sales_train.groupby('item_category_id', as_index=False).agg({"item_cnt_day": "sum"})['item_cnt_day'])
plt.xlabel("Item category", size = 15) 
# plt.ylabel("Sum of sales", size = 15) 
plt.title("Sum of sales per item category" , size = 20) 

**Some categories like "40" have way more sales than categories like "18"**

In [ ]:
df = sales_train.groupby(['date_block_num','region'],as_index=False)[['item_cnt_day']].sum()

In [ ]:
for i in df.date_block_num.unique():
    print(df[df['date_block_num'] == i].nlargest(5,'item_cnt_day'))

In [ ]:
sales_train.columns

In [ ]:
# sales_train[['date_block_num', 'shop_id', 'item_id', 'item_price', 'item_cnt_day',
#        'item_category_id', 'tot_item', 'region']].resample('M').agg(['sum'])

In [ ]:
sales_train.head()

In [ ]:
agg = sales_train.groupby(['date_block_num','shop_id', 'item_id'], as_index=False).agg({"item_price":"mean","item_cnt_day": "sum"})
agg

In [ ]:
agg = agg.join(shops, on = 'shop_id', lsuffix = '_caller',rsuffix = '_other')

In [ ]:
agg.drop(['shop_name','shop_id_other','name'],axis=1,inplace=True)
agg.rename(columns={'shop_id_caller': 'shop_id'},
          inplace=True, errors='raise')

In [ ]:
agg = agg.join(items, on = 'item_id', lsuffix = '_caller',rsuffix = '_other')

In [ ]:
agg.drop(['item_name','item_id_other'],axis=1,inplace=True)
agg.rename(columns={'item_id_caller': 'item_id'},
          inplace=True, errors='raise')

In [ ]:
agg.head()

In [ ]:
agg = agg[['date_block_num','shop_id','item_id','item_price','region','item_category_id','item_cnt_day']]
agg

**We want to predict "item_cnt_day" for the coming month**

In [ ]:
agg

lagging features

In [ ]:
import sqlite3
with sqlite3.connect(":memory:") as conn:
    agg.to_sql(name="aggs", con=conn, index=False)


In [ ]:
ex1_sql_query = """
SELECT
    date_block_num,shop_id,item_id,item_price,region,item_category_id,item_cnt_day,
    lag(item_cnt_day) over ( partition by shop_id,item_id order by date_block_num) as lag1,
    lag(item_cnt_day,2) over ( partition by shop_id,item_id order by date_block_num) as lag2,
    lag(item_cnt_day,3) over ( partition by shop_id,item_id order by date_block_num) as lag3,
    lag(item_cnt_day,4) over ( partition by shop_id,item_id order by date_block_num) as lag4,
    lag(item_price) over ( partition by shop_id,item_id order by date_block_num) as lag_price,
    avg(item_cnt_day) over ( partition by shop_id,item_id order by date_block_num rows between unbounded preceding and 1 preceding ) as avg_lag,
    min(item_cnt_day) over ( partition by shop_id,item_id order by date_block_num rows between unbounded preceding and 1 preceding ) as min_lag,
    max(item_cnt_day) over ( partition by shop_id,item_id order by date_block_num rows between unbounded preceding and 1 preceding ) as max_lag


FROM
    aggs
order by shop_id,item_id,date_block_num
"""
ex1_sql = pd.read_sql(ex1_sql_query, con=conn)
ex1_sql

In [ ]:
add

In [ ]:
agg.loc[(agg.shop_id==agg.iloc[i,:].shop_id) & (agg.item_id==agg.iloc[i,:].item_id)] []

# 📏Baseline Models

**Lets start by creating a baseline model that simply uses the available features**